# C++ Implementation of $\bar y(t)$

## What Are We Trying To Compute? 

\begin{align*}
\bar y(t) &= {\color{orange} H}\left(\int_0^t {\color{orange} H^{-1} (H_f^{\mathsf T})^{-1}\sigma_f^0}D_fD_f^{\mathsf T} {\color{orange}\sigma_f^0 H_f^{-1} H^{-1}}\,ds \right){\color{orange} H}, \\
\sigma_f^0(t) &= 
\begin{bmatrix}
\lambda_1^f(t)(a_1^f(t) + b_1^f(t)f_1(0, t+\delta_1)) & 0 & 0\\
0 & \lambda_2^f(t)(a_2^f(t) + b_2^f(t)f_2(0, t+\delta_2)) & 0\\
0 & 0 & \lambda_3^f(t)(a_3^f(t) + b_3^f(t)f_3(0, t+\delta_3))\\
\end{bmatrix}\\
H(t) &= \begin{bmatrix}
e^{-\int_0^t\kappa_1(s)\,ds} &&\\
& e^{-\int_0^t\kappa_2(s)\,ds} &\\
&& e^{-\int_0^t\kappa_3(s)\,ds}
\end{bmatrix}
\end{align*}


## `Subfunction` Class

* Subclass of `std::function<dobule(double)>` with operators +-*/() and `exp` overloaded
* `f + 2*g*h`

## `PiecewiseFunction` Class

* Keep a vector of times and a vector of subfunctions
* Also has operators +-*/() and `exp` overloaded
* Constructors for
    * General piecewise function
    * Piecewise constant function
    * Integral of a piecewise function

```c++
std::vector<double> times = {0.09, 0.25, 0.5, 1, 2, 3, 5, 10, 20, 30};

std::vector<double> k1 = {0.015, 0.02, 0.025, 0.027, 0.028, 0.029, 0.03, 0.03, 0.03, 0.03};
std::vector<double> k2 = {0.15, 0.2, 0.25, 0.27, 0.28, 0.29, 0.3, 0.3, 0.3, 0.3}; 
std::vector<double> k3 = {0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.6, 0.6, 0.6};

PiecewiseFunction k1_int(times, k1, true);
PiecewiseFunction k2_int(times, k1, true);
PiecewiseFunction k3_int(times, k1, true);

PiecewiseFunction h1 = exp(-k1_int);
PiecewiseFunction h2 = exp(-k2_int);
PiecewiseFunction h3 = exp(-k3_int); 
```
   

## `MatrixPiecewiseFunction` Class

* Keep a matrix of piecewise functions
* overloaded +-*/() and
    * transpose
    * inverse (closed form by cofactors)
    * integral

```c++
MatrixPiecewiseFunction H, H_f, Lambda, A, B, F; 
MatrixPiecewiseFunction sigma_f_0 = Lambda*(A + B*F);
std::vector<double> DDT;

double y_bar(const double& t)
{
    return (H*( H.inv() * H_f.T().inv() * sigma_f_0 * DDT * sigma_f_0 * H_f.inv() * H.inv() ).integral() * H)(t);
}
```

## Source Code

In [2]:
#include <functional>
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm>

class Subfunction {
public:
    Subfunction(std::function<double(double)> func) : m_func(func) {}

    double operator()(double t) const {
        return m_func(t);
    }
    Subfunction operator+(const Subfunction& other) const {
        return Subfunction([=](double t) { return m_func(t) + other.m_func(t); });
    }
    Subfunction operator-(const Subfunction& other) const {
        return Subfunction([=](double t) { return m_func(t) - other.m_func(t); });
    }
    Subfunction operator*(const Subfunction& other) const {
        return Subfunction([=](double t) { return m_func(t) * other.m_func(t); });
    }
    Subfunction operator/(const Subfunction& other) const {
        return Subfunction([=](double t) { return m_func(t) / other.m_func(t); });
    }
    Subfunction operator-() const {
        return Subfunction([=](double t) { return -m_func(t); });
    }
    friend Subfunction operator*(double scalar, const Subfunction& func) {
        return Subfunction([=](double t) { return scalar * func.m_func(t); });
    }
    friend Subfunction operator*(const Subfunction& func, double scalar) {
        return Subfunction([=](double t) { return func.m_func(t) * scalar; });
    }
private:
    std::function<double(double)> m_func;
};


Subfunction f([](double t) { return t * t; });
Subfunction g([](double t) { return t + 2; });

std::cout << (-(f + g))(3);

-14

In [3]:
class PiecewiseFunction {
public:
    PiecewiseFunction() {}
    PiecewiseFunction(const std::vector<double>& times, const std::vector<Subfunction>& functions)
        : m_times(times), m_functions(functions) {}

    // convient constructor for piecewise constant function or integral of a piecewise constant function
    PiecewiseFunction(const std::vector<double>& times, const std::vector<double>& constants, bool integrate = false) {
        m_times = times;
        if (integrate) {
            // Compute the integral of the piecewise constant function
            std::vector<Subfunction> integral_functions;

            for (size_t i = 0; i < times.size(); ++i) {
                Subfunction integral_func([=](double t) {
                    size_t q_t = std::upper_bound(times.begin(), times.end(), t) - times.begin() + 1;
                    double integral_value = 0.0;

                    // Add the sum of the intervals [T_j, T_{j+1}] for j from 0 to q(t)-1
                    for (size_t j = 0; j < q_t ; ++j) {
                        integral_value += constants[j] * (times[j] - (j==0 ? 0 : times[j-1]));
                    }
                    // Add the remaining term -g_{q(t)-1} * (T_{q(t)} - t)
                    integral_value -= constants[q_t - 1] * (times[q_t - 1] - t);
                    
                    return integral_value;
                });

                integral_functions.push_back(integral_func);
            }
            m_functions = integral_functions;
        } else {
            // Initialize the piecewise constant functions
            for (const auto& constant : constants) {
                m_functions.emplace_back([constant](double) { return constant; });
            }
        }
    }

    double operator()(double t) const {
        for (size_t i = 0; i < m_times.size(); ++i) {
            if (t < m_times[i]) {
                return m_functions[i](t);
            }
        }
        return m_functions.back()(t);  // use the last subfunction if t > T_N
    }

    PiecewiseFunction operator+(const PiecewiseFunction& other) const {
        std::vector<Subfunction> new_functions;
        for (size_t i = 0; i < m_functions.size(); ++i) {
            new_functions.push_back(m_functions[i] + other.m_functions[i]);
        }
        return PiecewiseFunction(m_times, new_functions);
    }

    PiecewiseFunction& operator+=(const PiecewiseFunction& other) {
        for (size_t i = 0; i < m_functions.size(); ++i) {
            m_functions[i] = m_functions[i] + other.m_functions[i];
        }
        return *this;
    }

    PiecewiseFunction operator-(const PiecewiseFunction& other) const {
        std::vector<Subfunction> new_functions;
        for (size_t i = 0; i < m_functions.size(); ++i) {
            new_functions.push_back(m_functions[i] - other.m_functions[i]);
        }
        return PiecewiseFunction(m_times, new_functions);
    }

    PiecewiseFunction operator*(const PiecewiseFunction& other) const {
        std::vector<Subfunction> new_functions;
        for (size_t i = 0; i < m_functions.size(); ++i) {
            new_functions.push_back(m_functions[i] * other.m_functions[i]);
        }
        return PiecewiseFunction(m_times, new_functions);
    }

    PiecewiseFunction operator/(const PiecewiseFunction& other) const {
        std::vector<Subfunction> new_functions;
        for (size_t i = 0; i < m_functions.size(); ++i) {
            new_functions.push_back(m_functions[i] / other.m_functions[i]);
        }
        return PiecewiseFunction(m_times, new_functions);
    }
    PiecewiseFunction operator-() const {
        std::vector<Subfunction> new_functions;
        for (const auto& func : m_functions) {
            new_functions.push_back(-func);
        }
        return PiecewiseFunction(m_times, new_functions);
    }

    friend PiecewiseFunction operator*(double scalar, const PiecewiseFunction& pf) {
        std::vector<Subfunction> new_functions;
        for (const auto& func : pf.m_functions) {
            new_functions.push_back(scalar * func);
        }
        return PiecewiseFunction(pf.m_times, new_functions);
    }

    friend PiecewiseFunction operator*(const PiecewiseFunction& pf, double scalar) {
        std::vector<Subfunction> new_functions;
        for (const auto& func : pf.m_functions) {
            new_functions.push_back(func * scalar);
        }
        return PiecewiseFunction(pf.m_times, new_functions);
    }

    friend PiecewiseFunction exp(const PiecewiseFunction& pf);

private:
    std::vector<double> m_times;
    std::vector<Subfunction> m_functions;
};

// Define some Subfunctions
Subfunction f1([](double t) { return t * t; });
Subfunction f2([](double t) { return 2 * t + 1; });
Subfunction f3([](double t) { return t - 3; });

// Define time intervals (without initial time 0)
std::vector<double> times = {1, 2, 3};

// Define PiecewiseFunctions
PiecewiseFunction pwf1(times, {f1, f2, f3});
PiecewiseFunction pwf2(times, {f2, f3, f1});

// Evaluate PiecewiseFunctions at 1.5
((-(pwf1 + pwf2))*2)(1.5)

-5.0000000

In [4]:
Subfunction exp(const Subfunction& f) {
    return Subfunction([=](double t) {
        return std::exp(f(t));
    });
}

In [5]:
PiecewiseFunction exp(const PiecewiseFunction& pf) {
    std::vector<Subfunction> new_functions;
    for (const auto& func : pf.m_functions) {
        new_functions.push_back(exp(func));
    }
    return PiecewiseFunction(pf.m_times, new_functions);
}

In [6]:
// Define g as the integral of a piecewise constant function

std::vector<double> times = {2, 4, 8};
std::vector<double> constants = {4, 5, 7};

PiecewiseFunction g(times, constants, true);

(std::vector<double>){g(0.5), g(2.5), g(5)}

{ 2.0000000, 10.500000, 25.000000 }

In [7]:
// exp working correctly

(std::vector<double>){exp(g)(0.5), exp(g)(2.5), exp(g)(5)}

{ 7.3890561, 36315.503, 7.2004899e+10 }

In [8]:
(std::vector<double>){exp(g(0.5)), exp(g(2.5)), exp(g(5))}

{ 7.3890561, 36315.503, 7.2004899e+10 }

In [9]:
std::vector<double> times = {0.09, 0.25, 0.5, 1, 2, 3, 5, 10, 20, 30};
std::vector<double> zeros = {0, 0, 0, 0, 0, 0, 0, 0, 0, 0};

std::vector<double> k1 = {0.015, 0.02, 0.025, 0.027, 0.028, 0.029, 0.03, 0.03, 0.03, 0.03};
std::vector<double> k2 = {0.15, 0.2, 0.25, 0.27, 0.28, 0.29, 0.3, 0.3, 0.3, 0.3}; 
std::vector<double> k3 = {0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.6, 0.6, 0.6};

std::vector<double> lambda1 = {0.015, 0.02, 0.025, 0.027, 0.028, 0.029, 0.03, 0.03, 0.03, 0.03};
std::vector<double> lambda2 = {0.15, 0.2, 0.25, 0.27, 0.28, 0.29, 0.3, 0.3, 0.3, 0.3}; 
std::vector<double> lambda3 = {0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.6, 0.6, 0.6};

std::vector<double> a1 = {0.015, 0.02, 0.025, 0.027, 0.028, 0.029, 0.03, 0.03, 0.03, 0.03};
std::vector<double> a2 = {0.15, 0.2, 0.25, 0.27, 0.28, 0.29, 0.3, 0.3, 0.3, 0.3}; 
std::vector<double> a3 = {0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.6, 0.6, 0.6};

std::vector<double> b1 = {0.015, 0.02, 0.025, 0.027, 0.028, 0.029, 0.03, 0.03, 0.03, 0.03};
std::vector<double> b2 = {0.15, 0.2, 0.25, 0.27, 0.28, 0.29, 0.3, 0.3, 0.3, 0.3}; 
std::vector<double> b3 = {0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.6, 0.6, 0.6};

std::vector<double> f1 = {0.015, 0.02, 0.025, 0.027, 0.028, 0.029, 0.03, 0.03, 0.03, 0.03};
std::vector<double> f2 = {0.15, 0.2, 0.25, 0.27, 0.28, 0.29, 0.3, 0.3, 0.3, 0.3}; 
std::vector<double> f3 = {0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.6, 0.6, 0.6};

PiecewiseFunction zeroFunction(times, zeros);

PiecewiseFunction k1_int(times, k1, true);
PiecewiseFunction k2_int(times, k1, true);
PiecewiseFunction k3_int(times, k1, true);

PiecewiseFunction h1 = exp(-k1_int);
PiecewiseFunction h2 = exp(-k2_int);
PiecewiseFunction h3 = exp(-k3_int);

PiecewiseFunction h1_inv = exp(k1_int);
PiecewiseFunction h2_inv = exp(k2_int);
PiecewiseFunction h3_inv = exp(k3_int);

PiecewiseFunction lambda1_(times, lambda1);
PiecewiseFunction lambda2_(times, lambda2);
PiecewiseFunction lambda3_(times, lambda3);

PiecewiseFunction a1_(times, a1);
PiecewiseFunction a2_(times, a2);
PiecewiseFunction a3_(times, a3);

PiecewiseFunction b1_(times, b1);
PiecewiseFunction b2_(times, b2);
PiecewiseFunction b3_(times, b3);

PiecewiseFunction f1_(times, f1);
PiecewiseFunction f2_(times, f2);
PiecewiseFunction f3_(times, f3);

In [10]:
#include <functional>
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm>


class MatrixPiecewiseFunction {
public:
    // Constructor that takes a PiecewiseFunction as an input argument
    MatrixPiecewiseFunction(const PiecewiseFunction& pf) : pf_(pf) {
        // Initialize each element to the given PiecewiseFunction
        matrix.resize(3, std::vector<PiecewiseFunction>(3, pf_));
    }

    MatrixPiecewiseFunction cofactorMatrix() const {
        MatrixPiecewiseFunction cofactor(pf_);

        for (int i = 0; i < 3; ++i) {
            for (int j = 0; j < 3; ++j) {
                std::vector<std::vector<PiecewiseFunction>> minorMatrix(2, std::vector<PiecewiseFunction>(2));
                
                // Fill the minor matrix
                int minorRow = 0;
                for (int row = 0; row < 3; ++row) {
                    if (row == i) continue;
                    int minorCol = 0;
                    for (int col = 0; col < 3; ++col) {
                        if (col == j) continue;
                        minorMatrix[minorRow][minorCol] = matrix[row][col];
                        minorCol++;
                    }
                    minorRow++;
                }
                
                // Compute the determinant of the minor matrix
                PiecewiseFunction minorDet = minorMatrix[0][0] * minorMatrix[1][1] - minorMatrix[0][1] * minorMatrix[1][0];
                
                // Compute the cofactor
                cofactor.matrix[i][j] = ((i + j) % 2 == 0 ? 1 : -1) * minorDet;
            }
        }

        return cofactor;
    }

    PiecewiseFunction determinant() const {
        MatrixPiecewiseFunction cofactor = this->cofactorMatrix();

        // Compute the determinant using the first row
        PiecewiseFunction det = matrix[0][0] * cofactor.matrix[0][0] +
                                matrix[0][1] * cofactor.matrix[0][1] +
                                matrix[0][2] * cofactor.matrix[0][2];
        
        return det;
    }

    MatrixPiecewiseFunction inverse() const {
        PiecewiseFunction det = this->determinant();
        // assuming determinant is not zero)
        MatrixPiecewiseFunction cofactor = this->cofactorMatrix();
        
        // Create a MatrixPiecewiseFunction from the transposed cofactor matrix
        MatrixPiecewiseFunction adjugate = this->transpose();
        MatrixPiecewiseFunction inverse(pf_);
        for (int i = 0; i < 3; ++i) {
            for (int j = 0; j < 3; ++j) {
                inverse.matrix[i][j] = adjugate.matrix[i][j] / det;
            }
        }
        return inverse;
    }

    MatrixPiecewiseFunction transpose() const {
        MatrixPiecewiseFunction transposed(pf_);
        for (int i = 0; i < 3; ++i) {
            for (int j = 0; j < 3; ++j) {
                transposed.matrix[i][j] = matrix[j][i];
            }
        }
        return transposed;
    }

    MatrixPiecewiseFunction operator+(const MatrixPiecewiseFunction& other) const {
        MatrixPiecewiseFunction result(pf_);
        for (int i = 0; i < 3; ++i) {
            for (int j = 0; j < 3; ++j) {
                result.matrix[i][j] = matrix[i][j] + other.matrix[i][j];
            }
        }
        return result;
    }

    MatrixPiecewiseFunction operator-(const MatrixPiecewiseFunction& other) const {
        MatrixPiecewiseFunction result(pf_);
        for (int i = 0; i < 3; ++i) {
            for (int j = 0; j < 3; ++j) {
                result.matrix[i][j] = matrix[i][j] - other.matrix[i][j];
            }
        }
        return result;
    }

    MatrixPiecewiseFunction operator*(const MatrixPiecewiseFunction& other) const {
        MatrixPiecewiseFunction result(pf_);
        for (int i = 0; i < 3; ++i) {
            for (int j = 0; j < 3; ++j) {
                for (int k = 0; k < 3; ++k) {
                    result.matrix[i][j] = result.matrix[i][j] + matrix[i][k] * other.matrix[k][j];
                }
            }
        }
        return result;
    }

    std::vector<std::vector<double>> evaluate(const double& t) const {
        std::vector<std::vector<double>> evaluated(3, std::vector<double>(3));
        for (int i = 0; i < 3; ++i) {
            for (int j = 0; j < 3; ++j) {
                evaluated[i][j] = matrix[i][j](t);
            }
        }
        return evaluated;
    }

    void printEvaluated(const double& t) const {
        std::vector<std::vector<double>> res = this->evaluate(t);
        for (size_t i = 0; i < 3; ++i) {
            for (size_t j = 0; j < 3; ++j) {
                std::cout << res[i][j] << ", ";
            }
            std::cout << std::endl;
        }
    }

    // Overload operator[] to access rows and elements
    std::vector<PiecewiseFunction>& operator[](size_t index) {
        return matrix[index];
    }

    const std::vector<PiecewiseFunction>& operator[](size_t index) const {
        return matrix[index];
    }

private:
    std::vector<std::vector<PiecewiseFunction>> matrix;
    PiecewiseFunction pf_;
};


In [11]:
#include <iostream>
#include <vector>

// Assuming zeroFunction, h1, h2, h3, a1_, a2_, a3_, b1_, b2_, b3_, f1_, f2_, f3_, lambda1_, lambda2_, lambda3_
// are predefined instances of PiecewiseFunction

MatrixPiecewiseFunction H(zeroFunction), Lambda(zeroFunction), A(zeroFunction), B(zeroFunction), F(zeroFunction);

// Initialize DDT using std::vector<std::vector<double>>
std::vector<std::vector<double>> DDT(3, std::vector<double>(3, 0.0));

DDT[0][0] = 1.00; DDT[0][1] = 0.85; DDT[0][2] = 0.75;
DDT[1][0] = 0.85; DDT[1][1] = 1.00; DDT[1][2] = 0.65;
DDT[2][0] = 0.75; DDT[2][1] = 0.65; DDT[2][2] = 1.00;

H[0][0] = h1;
H[1][1] = h2;
H[2][2] = h3;

A[0][0] = a1_;
A[1][1] = a2_;
A[2][2] = a3_;

B[0][0] = b1_;
B[1][1] = b2_;
B[2][2] = b3_;

F[0][0] = f1_;
F[1][1] = f2_;
F[2][2] = f3_;

Lambda[0][0] = lambda1_;
Lambda[1][1] = lambda2_;
Lambda[2][2] = lambda3_;

// Evaluate (A + B) at t = 3.5
(A + B).printEvaluated(3.5);


0.06, 0, 0, 
0, 0.6, 0, 
0, 0, 1.2, 


In [12]:
#pragma cling add_include_path("/usr/include/eigen3/")

#include <Eigen/Dense>
#include <iostream>
#include <stdexcept>

Eigen::MatrixXd inverse(const Eigen::MatrixXd& matrix) {
    size_t n = matrix.rows();
    Eigen::MatrixXd cofactor(n, n);

    for (size_t i = 0; i < n; ++i) {
        for (size_t j = 0; j < n; ++j) {
            Eigen::MatrixXd _minor(n - 1, n - 1);
            for (size_t row = 0; row < n; ++row) {
                if (row == i) continue;
                size_t minorRow = (row < i) ? row : row - 1;
                for (size_t col = 0; col < n; ++col) {
                    if (col == j) continue;
                    size_t minorCol = (col < j) ? col : col - 1;
                    _minor(minorRow, minorCol) = matrix(row, col);
                }
            }
            cofactor(i, j) = (i + j) % 2 == 0 ? _minor.determinant() : -_minor.determinant();
        }
    }

    return cofactor.transpose()/matrix.determinant();
}

Eigen::MatrixXd mat(3, 3);
mat << 4, 3, 2,
       3, 1, 1,
       2, 1, 3;

Eigen::MatrixXd res = inverse(mat)*mat;

for (size_t i=0 ; i<3 ; ++i){
    for (size_t j=0 ; j<3 ; ++j){
        std::cout << res(i, j) << ", ";
    }
    std::cout << std::endl;
}

1, 2.77556e-17, 0, 
1.11022e-16, 1, 0, 
0, 0, 1, 
